# Binary Classification Problem

In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [4]:
bc = datasets.load_breast_cancer()
x,y = bc.data, bc.target
n_samples, n_features = x.shape
print(n_samples,n_features)

569 30


In [5]:
x_train,x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=1234)

In [6]:
# scale the featurers